**Changing the directory**

In [1]:
cd '/content/drive/MyDrive/NasrAbadi/Miss-Match detection/StratifiedGroupFold'

/content/drive/.shortcut-targets-by-id/1QjZ52EqSaXzJIHNRgjdB-qbPUcReg_SP/NasrAbadi/Miss-Match detection/StratifiedGroupFold


**Loading the data**

In [2]:
import numpy as np
X = np.load('X.npy')
Y = np.load('Y.npy')

print("X Shape:",X.shape)
print("Y shape:",Y.shape)
print(Y)
import scipy.io
SegmentsPerSubject = scipy.io.loadmat('SegmentsPerSubject', mat_dtype=True)
SegmentsPerSubject = SegmentsPerSubject['SegmentsPerSubject']
SegmentsPerSubject = np.array(SegmentsPerSubject)
SegmentsPerSubject = SegmentsPerSubject.transpose()
print("shape Segments:", SegmentsPerSubject.shape)

Group =[]
for i in range(SegmentsPerSubject.shape[0]):
  for j in range(int(SegmentsPerSubject[i])):
    Group.append(i)
Group = np.array(Group)
Group = np.expand_dims(Group, axis=1)
print(Group)
print("Group shape is:",Group.shape)

X Shape: (7916, 1, 19, 1280)
Y shape: (7916,)
[0 0 0 ... 1 1 1]
shape Segments: (121, 1)
[[  0]
 [  0]
 [  0]
 ...
 [120]
 [120]
 [120]]
Group shape is: (7916, 1)


**Installing Required Packages**

In [3]:
!pip install scikeras
!pip install pyriemann
!pip install scikit-optimize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 KB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyriemann: filename=pyriemann-0.4-py2.py3-none-any.whl size=105118 sha256=b97d93b8526f9c2f782086665d8355795e57b7eeffd047f92a6f8b1173b3c580
  Stored in directory: /root/.cache/pip/wheels/1a/32/b8/6f844cc26ba42daee387f6aacc4b80f128233ff84703380967
Successfully built pyriemann
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 KB 11.1 MB/s eta 0:00:00


**Change the channels to first**

In [4]:
from tensorflow.keras import backend as K
K.set_image_data_format('channels_first')

**Building the Model**

In [5]:
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras import utils as np_utils
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import StratifiedGroupKFold, GridSearchCV
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import Adam
from keras.models import load_model
import matplotlib.pyplot as plt
from sklearn.metrics import fbeta_score
import math
from numpy import mean
from numpy import std
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.model_selection import cross_val_score

def create_model(kernelLength = 32, nb_classes = 1, Chans = 19, Samples = 1280, 
            dropoutRate = 0.5 , F1 = 8, 
            D = 2, F2 = 16, norm_rate = 0.25, dropoutType = 'Dropout'):

  if dropoutType == 'SpatialDropout2D':
      dropoutType = SpatialDropout2D
  elif dropoutType == 'Dropout':
      dropoutType = Dropout
  else:
      raise ValueError('dropoutType must be one of SpatialDropout2D '
                        'or Dropout, passed as a string.')
  
  input1   = Input(shape = (1, Chans, Samples))

  ##################################################################
  block1       = Conv2D(F1, (1, kernelLength), padding = 'same',
                                  input_shape = (1, Chans, Samples),
                                  use_bias = False)(input1)
  block1       = BatchNormalization(axis = 1)(block1)
  block1       = DepthwiseConv2D((Chans, 1), use_bias = False, 
                                  depth_multiplier = D,
                                  depthwise_constraint = max_norm(1.))(block1)
  block1       = BatchNormalization(axis = 1)(block1)
  block1       = Activation('elu')(block1)
  block1       = AveragePooling2D((1, 4))(block1)
  block1       = dropoutType(dropoutRate)(block1)
  
  block2       = SeparableConv2D(F2, (1, 16),
                                  use_bias = False, padding = 'same')(block1)
  block2       = BatchNormalization(axis = 1)(block2)
  block2       = Activation('elu')(block2)
  block2       = AveragePooling2D((1, 8))(block2)
  block2       = dropoutType(dropoutRate)(block2)
      
  flatten      = Flatten(name = 'flatten')(block2)
  
  dense        = Dense(nb_classes, name = 'dense', 
                        kernel_constraint = max_norm(norm_rate))(flatten)
  sigmoid      = Activation('sigmoid', name = 'sigmoid')(dense)
  
  model        = Model(inputs=input1, outputs=sigmoid)
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy'])
  return model

**Spliting the data to early stopping and test dataset**

In [6]:
group_kfold1 = StratifiedGroupKFold(n_splits=10, shuffle=True, random_state=2022)
# Test split ....
split_test = group_kfold1.split(X,Y,Group)
d_test = next(split_test)
X_tr = X[d_test[0]]
Y_tr = Y[d_test[0]]
X_test = X[d_test[1]]
Y_test = Y[d_test[1]]
Group_tr = Group[d_test[0]]
Group_test = Group[d_test[1]]
# Early stopping spit ...
group_kfold2 = StratifiedGroupKFold(n_splits=18, shuffle=True, random_state=2022)
split_early = group_kfold2.split(X_tr,Y_tr,Group_tr)
d_early = next(split_early)
X_train = X_tr[d_early[0]]
Y_train = Y_tr[d_early[0]]
X_early = X_tr[d_early[1]]
Y_early = Y_tr[d_early[1]]
Group_train = Group_tr[d_early[0]]
Group_early = Group_tr[d_early[1]]

del X,Y,X_tr,Y_tr, d_early,split_early,d_test, split_test, Group, Group_tr
### Compute Class_weight ###
class_weights = compute_class_weight(class_weight = "balanced",classes = np.unique(Y_train), y = Y_train)
class_weights = dict(zip(np.unique(Y_train), class_weights))
class_weights[0] = int(10*round(class_weights[0],1))
class_weights[1] = int(10*round(class_weights[1],1))
class_weight = class_weights
#class_weight = [{0: 1, 1: class_weights[0]}, {0: 1, 1: class_weights[1]}]
print("The Class Weight is:",class_weight)

The Class Weight is: {0: 9, 1: 11}


In [7]:
print("X_train shape:",X_train.shape)
print("X_early shape:",X_early.shape)
print("X_test shape:",X_test.shape)
print("----------------------------")
print("Y_train shape:",Y_train.shape)
print("Y_early shape:",Y_early.shape)
print("Y_test shape:",Y_test.shape)
print('----------------------------')
print("Group_train shape:",Group_train.shape)
print("Group_early shape:",Group_early.shape)
print("Group_test shape:",Group_test.shape)

X_train shape: (6699, 1, 19, 1280)
X_early shape: (504, 1, 19, 1280)
X_test shape: (713, 1, 19, 1280)
----------------------------
Y_train shape: (6699,)
Y_early shape: (504,)
Y_test shape: (713,)
----------------------------
Group_train shape: (6699, 1)
Group_early shape: (504, 1)
Group_test shape: (713, 1)


**Initiating the instance of the model, Organizing the callbacks and using the GridSearchCV**

In [8]:

### Callbacks ###
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=15)
callbacks = [es]
#### KerasClassifier Defining ####
model = KerasClassifier(model=create_model, verbose = 2,shuffle=True,
                        epochs=100,batch_size=32,fit__validation_data =(X_early, Y_early),
                        class_weight=class_weight,callbacks=callbacks)
# define the grid search parameters
kernelLength  = [32, 64]
param_grid = dict(model__kernelLength=kernelLength)
print(param_grid)
grid = GridSearchCV(estimator=model,param_grid=param_grid, cv=2)
grid_result = grid.fit(X_train, y=Y_train, groups=Group_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

{'model__kernelLength': [32, 64]}
Epoch 1/100
105/105 - 15s - loss: 5.2216 - accuracy: 0.7375 - val_loss: 0.8903 - val_accuracy: 0.3889 - 15s/epoch - 141ms/step
Epoch 2/100
105/105 - 1s - loss: 3.8500 - accuracy: 0.8471 - val_loss: 0.7842 - val_accuracy: 0.5794 - 1s/epoch - 12ms/step
Epoch 3/100
105/105 - 1s - loss: 2.3807 - accuracy: 0.9307 - val_loss: 0.8286 - val_accuracy: 0.5278 - 1s/epoch - 13ms/step
Epoch 4/100
105/105 - 1s - loss: 1.1844 - accuracy: 0.9806 - val_loss: 0.7449 - val_accuracy: 0.5476 - 1s/epoch - 10ms/step
Epoch 5/100
105/105 - 1s - loss: 0.7016 - accuracy: 0.9964 - val_loss: 0.4953 - val_accuracy: 0.8274 - 984ms/epoch - 9ms/step
Epoch 6/100
105/105 - 1s - loss: 0.5568 - accuracy: 0.9952 - val_loss: 0.3949 - val_accuracy: 0.8631 - 1s/epoch - 10ms/step
Epoch 7/100
105/105 - 1s - loss: 0.3830 - accuracy: 0.9994 - val_loss: 0.3555 - val_accuracy: 0.9028 - 1s/epoch - 10ms/step
Epoch 8/100
105/105 - 1s - loss: 0.3116 - accuracy: 1.0000 - val_loss: 0.3216 - val_accuracy: